In [1]:
!git clone https://github.com/gmonteiroc21/desafio_bookbridge.git

Cloning into 'desafio_bookbridge'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 115 (delta 52), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 53.47 KiB | 1.27 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
%cd desafio_bookbridge

/content/desafio_bookbridge


###1- Importação e Limpeza

In [3]:
#importação do dataset seguindo as instruções na pagina

import kagglehub

# Download latest version
data_path = kagglehub.dataset_download("pypiahmad/goodreads-book-reviews1")

print("Path to dataset files:", data_path)

100%|██████████| 8.14G/8.14G [05:36<00:00, 26.0MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1


In [20]:
import pandas as pd
import json
import os
import sys
sys.path.append(os.path.abspath("scripts"))
from scripts.manipulating_data import *
from scripts.data_analysis import *
from scripts.data_preparation import *
from scripts.recommender import *

file_path = "/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/"


In [5]:
#transforma as interações dos usuarios em um dataframe
interactions_df = pd.read_csv(file_path+'goodreads_interactions.csv', nrows=20000000) #limita em  2M de linhas
print(interactions_df.head(1))

   user_id  book_id  is_read  rating  is_reviewed
0        0      948        1       5            0


In [7]:
#transforma as informações dos livros em um dataframe
books_df = select_books(file_path+'goodreads_books.json')
print(books_df)


         book_id                                            authors  \
0        5333265              [{'author_id': '604031', 'role': ''}]   
1        1333909              [{'author_id': '626222', 'role': ''}]   
2        7327624               [{'author_id': '10333', 'role': ''}]   
3        6066819                [{'author_id': '9212', 'role': ''}]   
4         287140              [{'author_id': '149918', 'role': ''}]   
...          ...                                                ...   
199995  22027751               [{'author_id': '68900', 'role': ''}]   
199996  22027750  [{'author_id': '205', 'role': ''}, {'author_id...   
199997  26333472  [{'author_id': '9311365', 'role': 'Editor'}, {...   
199998    839275               [{'author_id': '69826', 'role': ''}]   
199999  10302363              [{'author_id': '133666', 'role': ''}]   

                                                    title average_rating  \
0                             W.C. Fields: A Life on Film           4.0

In [ ]:
print(books_df)


In [ ]:
valid_books = interactions_df['book_id'].isin(books_df['book_id'])
interactions_read = interactions_df[(interactions_df['is_read'] == 1) & valid_books]

user_total_books = interactions_read.groupby('user_id').size().reset_index(name='user_total_books')

print(user_total_books)

tratamento do *books_df*

In [10]:
first_df = clean_empty_rows(books_df, 'book_id')

'''books_df = duplicate_size(file_path+'goodreads_books.json', books_df, colunas_desejadas)'''

books_df['average_rating'] = pd.to_numeric(first_df['average_rating'], errors='coerce')

print(books_df.head(1))

   book_id                                authors  \
0  5333265  [{'author_id': '604031', 'role': ''}]   

                         title  average_rating ratings_count num_pages  \
0  W.C. Fields: A Life on Film             4.0             3       256   

  publication_year text_reviews_count  
0             1984                  1  


In [ ]:
#transforma as informações dos autores em um dataframe
authors_df = pd.read_json(file_path+'goodreads_book_authors.json', lines=True)
print(authors_df.head(1))

In [6]:
#transforma as informações de generos em um dataframe
genres_df = pd.read_json(file_path+'goodreads_book_genres_initial.json', lines=True)
print(genres_df.head(10))


    book_id                                             genres
0   5333265      {'history, historical fiction, biography': 1}
1   1333909  {'fiction': 219, 'history, historical fiction,...
2   7327624  {'fantasy, paranormal': 31, 'fiction': 8, 'mys...
3   6066819  {'fiction': 555, 'romance': 23, 'mystery, thri...
4    287140                                 {'non-fiction': 3}
5    287141  {'history, historical fiction, biography': 9, ...
6    378460                                     {'fiction': 2}
7   6066812  {'fantasy, paranormal': 32, 'young-adult': 8, ...
8  34883016                                     {'romance': 3}
9    287149  {'non-fiction': 24, 'history, historical ficti...


In [ ]:
books_df = books_df.merge(
    genres_df[['book_id', 'genres']],  # Seleciona apenas 'book_id' e 'genres'
    on='book_id',  # A chave de junção
    how='left'  # Mantém todos os livros, mesmo aqueles sem gênero
)

### 2 - Análise exploratória e preparação dos dados para modelagem

In [8]:
unique_genres = ['history, historical fiction, biography', 'fiction', 'fantasy, paranormal'
 ,'mystery, thriller, crime', 'poetry', 'romance', 'non-fiction', 'children',
 'young-adult', 'comics, graphic']


In [11]:
genre_stats_df = genre_stats(genres_df, books_df, unique_genres)
print(genre_stats_df)

         book_id                                  genres  average_rating  \
442139  23006706  history, historical fiction, biography             5.0   
54752   16061666                                 fiction             5.0   
79068   31344082                     fantasy, paranormal             5.0   
221166  16127783                mystery, thriller, crime             5.0   
259500  34372505                                  poetry             5.0   
13971   36217797                                 romance             5.0   
449618  35389399                             non-fiction             5.0   
21167   36318802                                children             5.0   
332132  25414737                             young-adult             5.0   
118250  29004052                         comics, graphic             5.0   

        ratings_count                                      title  
442139             23                               Fate Twister  
54752              13  Explor

In [12]:
top_genre_per_user = calculate_top_genre_per_user(interactions_df, genres_df,books_df)
print(top_genre_per_user.head(5))

   user_id                          favorite_genre  genre_count  \
0        0                                 fiction           17   
1        1  history, historical fiction, biography            3   
2        2                                 fiction            3   
3        3  history, historical fiction, biography            5   
4        4                                 fiction            7   

   total_books_read  
0                21  
1                 4  
2                 5  
3                 5  
4                 9  


In [ ]:
print(top_genre_per_user.head(5))

In [13]:
user_literary_age = calculate_user_literary_age(interactions_df, books_df)
user_literary_age

,user_id,70-,70-00,00+
0,0,0,3,12
1,1,0,0,3
2,2,0,1,3
3,3,1,3,1
4,4,0,4,4
...,...,...,...,...
36587,39219,1,2,4
36588,39220,0,2,7
36589,39221,0,3,1
36590,39222,0,2,0


### Sugestão de livro e probabilidade de boa avaliação

In [15]:
user_id = int(input("Digite o seu ID de usuário: "))
n_books_user = top_genre_per_user[top_genre_per_user['user_id'] == user_id]['total_books_read'].values[0]
best_book, user_genre = recommend_books(user_id, top_genre_per_user, genre_stats_df)
print(f'Levando em consideração que seu gênero mais lido é o: {user_genre}\nVocê deve curtir o {best_book}')


Digite o seu ID de usuário: 2
Levando em consideração que seu gênero mais lido é o: fiction
Você deve curtir o Exploring the Garden with the Little Rose


In [22]:
book_id = int(input('Digite o ID do livro escolhido: '))

book_genre = get_book_genre(book_id, genres_df)
genre_count = user_genre_count(interactions_df, user_id)
time_period_books_read = time_range(books_df, book_id)

Digite o ID do livro escolhido: 54


TypeError: user_genre_count() takes 0 positional arguments but 2 were given

In [ ]:
# Calcula as proporções
genre_proportion = genre_count / n_books_user if n_books_user else 0
time_proportion = time_period_books_read / n_books_user if n_books_user else 0

# Calcula a pontuação final com os pesos
affinity_score = ((genre_proportion * 2) + (time_proportion ))/3

# Normaliza a pontuação para que seja uma porcentagem
affinity_score = min(max(affinity_score, 0), 1) * 100  # Garante que a pontuação seja entre 0 e 100

print(affinity_score)